In [31]:
import pandas as pd
import pathlib as pl
import collections as col

df = pd.read_csv(
    pl.Path(
        "/home/ebertp/work/code/marschall-lab/project-male-assembly/data",
        "sample_pop_color.v2.tsv")
    , sep="\t",
)

dest_file = pl.Path(
    "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/annotations/projectmng",
    "pop_colors.tsv"
)

df.drop(
    ["sample_num", "sample", "is_in_paper", "data_source", "Y_haplogroup", "workflow_sample"],
    axis=1, inplace=True
)

df.drop_duplicates("super_population", keep="first", inplace=True)

# source: https://convertacolor.com/
cmyk_map = {
    "D8A105": {
        "cmyk_cyan": 0,
        "cmyk_magenta": 25,
        "cmyk_yellow": 98,
        "cmyk_black": 15
    },
    "778500": {
        "cmyk_cyan": 11,
        "cmyk_magenta": 0,
        "cmyk_yellow": 100,
        "cmyk_black": 48
    },
    "710027": {
        "cmyk_cyan": 0,
        "cmyk_magenta": 100,
        "cmyk_yellow": 65,
        "cmyk_black": 56
    },
    "036C77": {
        "cmyk_cyan": 97,
        "cmyk_magenta": 9,
        "cmyk_yellow": 0,
        "cmyk_black": 53
    },
    "C44CFD": {
        "cmyk_cyan": 23,
        "cmyk_magenta": 70,
        "cmyk_yellow": 0,
        "cmyk_black": 1
    },
    
}


new_df = []
for row in df.itertuples():
    
    colors = col.OrderedDict({
        "continental_group": row.super_population,
        "color_name": row.superpop_color_name,
        "color_hex": row.superpop_color_hex.strip("#"),
        "rgb_red_rel": round(row.superpop_color_red, 3),
        "rgb_red_abs": int(round(row.superpop_color_red * 255)),
        "rgb_green_rel": round(row.superpop_color_green, 3),
        "rgb_green_abs": int(round(row.superpop_color_green * 255)),
        "rgb_blue_rel": round(row.superpop_color_blue, 3),
        "rgb_blue_abs": int(round(row.superpop_color_blue * 255))  
    })
    colors["rgb_rel"] = (
        f"{colors['rgb_red_rel']},"
        f"{colors['rgb_green_rel']},"
        f"{colors['rgb_blue_rel']}"
    )
    colors["rgb_abs"] = (
        f"{colors['rgb_red_abs']},"
        f"{colors['rgb_green_abs']},"
        f"{colors['rgb_blue_abs']}"
    )
    
    cmyk = cmyk_map[colors["color_hex"]]
    colors.update(cmyk)
    colors["cmyk"] = (
        f"{colors['cmyk_cyan']},"
        f"{colors['cmyk_magenta']},"
        f"{colors['cmyk_yellow']},"
        f"{colors['cmyk_black']}"
    )
    
    new_df.append(colors)
    
color_df = pd.DataFrame.from_records(new_df)
color_df.sort_values("continental_group", inplace=True)

with open(dest_file, "w") as table:
    table.write("# color scheme used in chrY paper, Hallast et al.\n")
    color_df.to_csv(table, sep="\t", header=True, index=False)